In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *
import re
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [145]:
full = pd.read_csv('final_movies.csv').dropna()
selections = full.drop(columns=['Unnamed: 0','movieId','imdbID','Title','Year','Ratings','Released','Runtime','Plot','Poster','imdbVotes'])

In [146]:
searched = 'Pretty in Pink'
value = full[full['Title']==searched].index.values[0]
thing = selections.iloc[value]
knn = NearestNeighbors(n_neighbors=200)
knn.fit(selections)
arr = knn.kneighbors([thing], return_distance = False)

In [147]:
test = pd.DataFrame(index=range(0,200), columns=full.columns)
for i in arr:
    test=full.iloc[i]

In [122]:
# searched = 'Toy Story'
# value = full[full['Title']==searched].index.values[0]

In [148]:
# new_full = full[full['clusters'] == full['clusters'][value]].reset_index()
new_full = test
plots_arr = new_full['Plot'].to_numpy()
plots_l = list(plots_arr)

In [149]:
len(new_full)

200

In [150]:
corpus = []
for i in range(0, len(plots_l)):
    text = re.sub('[^a-zA-Z]', ' ', plots_l[i])
    text = text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    corpus.append(text)

In [151]:
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X = vectorizer.fit_transform(corpus)

In [152]:
my_tags = vectorizer.get_feature_names()

In [153]:
len(my_tags)

100

In [154]:
my_tag_matrix = pd.DataFrame(0, index=np.arange(len(plots_l)), columns=my_tags)

In [155]:
for i in range(0,len(my_tag_matrix)):
    for j in range(0,len(my_tags)):        
        if my_tags[j] in new_full['Plot'].iloc[i]:
            my_tag_matrix.iloc[i][my_tags.index(my_tags[j])] = 1

In [156]:
my_tag_matrix['imdbID'] = new_full['imdbID']

In [177]:
new_value = new_full[new_full['Title']==searched]

In [178]:
new_selections = my_tag_matrix.drop(columns = ['imdbID'])

In [179]:
thing = new_selections.iloc[new_value]

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [180]:
new_value

,Unnamed: 0,movieId,imdbID,Title,Year,Released,Runtime,Plot,Poster,Ratings,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,votes


In [169]:
knn = NearestNeighbors(n_neighbors=4)

In [170]:
knn.fit(new_selections)
arr = knn.kneighbors([thing], return_distance = False)

ValueError: cannot copy sequence with size 0 to array axis with dimension 100

In [137]:
results = []
for i in arr[0][1:4]:
    url = f'https://imdb.com/title/{new_full["imdbID"].iloc[i]}'
    plot= new_full['Plot'].iloc[i]
    title = new_full['Title'].iloc[i]
    poster = new_full['Poster'].iloc[i]
    temp_dict = {
        'title':title,
        'url':url,
        'plot':plot,
        'poster':poster
    }
    results.append(temp_dict)

In [138]:
results

[{'title': 'The Girl from Missouri',
  'url': 'https://imdb.com/title/tt0025173',
  'plot': 'Chorus girl Eadie is determined to marry a millionaire without sacrificing her virtue.',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMjM0MTczMzE3NF5BMl5BanBnXkFtZTgwODI5MjIyMjE@._V1_SX300.jpg'},
 {'title': 'It Could Happen to You',
  'url': 'https://imdb.com/title/tt0110167',
  'plot': 'A police officer promises to share his lottery ticket with a waitress in lieu of a tip.',
  'poster': 'https://m.media-amazon.com/images/M/MV5BYzMxOGUzOWEtMGUxMi00ZWM1LWE3ODgtNzc4OGVjZmJhMmFmXkEyXkFqcGdeQXVyODY0NzcxNw@@._V1_SX300.jpg'},
 {'title': 'Cream Lemon',
  'url': 'https://imdb.com/title/tt0472840',
  'plot': 'Live action movie based on the OVA anime series.',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMWI4MTQ5NWYtYTFmZC00NTMwLWJjYTktNTdiNTRmNjM2MDQwXkEyXkFqcGdeQXVyMjE4MTIyNDI@._V1_SX300.jpg'}]